In [87]:
from PIL import Image, ImageDraw, ImageFont  
import numpy as np
import matplotlib.pyplot as plt
from copy import copy, deepcopy
from scipy import interpolate
import cv2 
from scipy.integrate import quad
from scipy.misc import derivative
import sympy as sp
import math
from math import sqrt, pi
# from scipy import integrate
from sympy import integrate, Symbol 
import scipy.integrate as spi
import Augmentor as p
import os

In [161]:
# Parameters
font_type = 'C:/Windows/Fonts/Calibri/calibrib.ttf'
font_size = 49 # 49
character = 'A'
threshold_binarisation = 100

In [162]:
# Befehl muss zweimal ausgeführt werden (komisch)
%matplotlib qt
%matplotlib qt

In [179]:
plt.close('all')
size_y, size_x = 200, 200
img = Image.new('RGB', (size_y, size_x), color = (0, 0, 0))   # (50,50)  
fnt = ImageFont.truetype(font_type, font_size)
d = ImageDraw.Draw(img)
d.text((20,20), character, font=fnt, fill=(255, 255, 255))
# img.show() 
# img.save('pil_text_font.bmp')
img = np.array(img)
plt.figure('img')
plt.imshow(img)
x,y,z = np.shape(img)
# print(x,y,z)
img[img<threshold_binarisation] = 0
plt.figure('binarised')
plt.imshow(img)
print(type(img))

x, y, w, h = cv2.boundingRect(img[:,:,0])
print(x, y, w, h)
x_min = x-1
y_min = y-1
x_max = x+w
y_max = y+h

y_position = int((size_y-h)/2)
x_position = int((size_x-w)/2)
print(y_position,x_position)

img_centered = Image.new('RGB', (size_y, size_x), color = (0, 0, 0))   # (50,50)  
fnt = ImageFont.truetype(font_type, font_size)
d_centered = ImageDraw.Draw(img_centered)
d_centered.text((x_position,y_position-5), character, font=fnt, fill=(255, 255, 255))
# img.show() 
# img.save('pil_text_font.bmp')
img_centered = np.array(img_centered)
plt.figure('img_centered')
plt.imshow(img_centered)

img_centered_binarised = deepcopy(img_centered)
img_centered_binarised[img1<threshold_binarisation] = 0
plt.figure('img1_center_binarised')
plt.imshow(img_centered_binarised)
print(type(img_centered_binarised))

<class 'numpy.ndarray'>
21 26 28 31
84 86


IndexError: boolean index did not match indexed array along dimension 0; dimension is 200 but corresponding boolean dimension is 100

200 200 3
<class 'numpy.ndarray'>


In [5]:
def image_rect(img):
    img_rect = deepcopy(img)
    x, y, w, h = cv2.boundingRect(img_rect[:,:,0])
    # print(x, y, w, h)
    x_min = x-1
    y_min = y-1
    x_max = x+w
    y_max = y+h
    cv2.rectangle(img_rect, (x_min, y_min), (x_max, y_max), (0, 255, 0), 0)   # img wird überschrieben
    img_without_boundary = img_rect[y:y_max, x:x_max, :]
    return img_without_boundary

In [6]:
# img.astype(np.float32)
# img.astype(np.uint8)
# get the bounding rect
img_rect = deepcopy(img)
x, y, w, h = cv2.boundingRect(img_rect[:,:,0])
# print(x, y, w, h)
x_min = x-1
y_min = y-1
x_max = x+w
y_max = y+h
cv2.rectangle(img_rect, (x_min, y_min), (x_max, y_max), (0, 255, 0), 0)   # img wird überschrieben
plt.figure('img_rect')
plt.imshow(img_rect/255)

In [18]:
img_without_boundary = img_rect[y:y_max, x:x_max,:]
plt.figure('img_without_boundary')
plt.imshow(img_without_boundary/255)

TypeError: slice indices must be integers or None or have an __index__ method

In [8]:
# print(img_without_boundary)
y_img_without_boundary, x_img_without_boundary, z_img_without_boundary = np.shape(img_without_boundary)
# print(y_img_without_boundary, x_img_without_boundary, z_img_without_boundary)

In [9]:
tmp_x_min = int(-1*x_img_without_boundary/2)
tmp_x_max = int(np.ceil(x_img_without_boundary/2))
tmp_y_min = int(-1*y_img_without_boundary/2)
tmp_y_max = int(np.ceil(y_img_without_boundary/2))
# print(tmp_x_min)
# print(tmp_x_max)
# print(tmp_y_min)
# print(tmp_y_max)
x_mesh = np.arange(tmp_x_min, tmp_x_max, 1)
y_mesh = np.arange(tmp_y_min, tmp_y_max, 1)
# print(x_mesh)
# print(y_mesh)
xx, yy = np.meshgrid(x_mesh, np.flip(y_mesh,0), sparse=True)
yy = yy*1j
z = (yy+xx)
# print(z)
q = np.pi/180
c_meshgrid = 0.5*z.imag*1j + 2*z.real  # (z*np.cos(q)-np.sin(q)) / (z*np.sin(q)+np.cos(q*5))
# print(c_meshgrid)
y_meshgrid, x_meshgrid = c_meshgrid.shape
# print(y_meshgrid, x_meshgrid)
# c_meshgrid_reshape = c_meshgrid.reshape((y_meshgrid,x_meshgrid,1))

In [10]:
c_meshgrid_mask = deepcopy(c_meshgrid)
max_real_meshgrid_mask = int(np.max(c_meshgrid_mask.real))
max_imag_meshgrid_mask = int(np.max(c_meshgrid_mask.imag))
min_real_meshgrid_mask = int(np.min(c_meshgrid_mask.real))
min_imag_meshgrid_mask = int(np.min(c_meshgrid_mask.imag))
# print(max_real_meshgrid_mask)
# print(max_imag_meshgrid_mask)
# print(min_real_meshgrid_mask)
# print(min_imag_meshgrid_mask)
# print(c_meshgrid_mask.shape)

In [11]:
z_transform = np.array(np.zeros((max_imag_meshgrid_mask+1-min_imag_meshgrid_mask, max_real_meshgrid_mask-min_real_meshgrid_mask+1, 3)))
y_z_transform, x_z_transform, z_z_transform = z_transform.shape
# print(z_transform)
# print(y_z_transform, x_z_transform, z_z_transform)

In [12]:
for i in range(y_meshgrid):
    for j in range(x_meshgrid):
        for k in range(3):
            z_transform[int(max_imag_meshgrid_mask-c_meshgrid_mask[i,j].imag), int(c_meshgrid_mask[i,j].real-min_real_meshgrid_mask),:] = img_without_boundary[i,j,k]
        
#   Für Y-Achse => max-y
#   Für X-Achse => x-min

In [13]:
print(z_transform.shape)
print(img_without_boundary.shape)

(15, 55, 3)
(31, 28, 3)


In [14]:
plt.figure('transformation')
plt.imshow(z_transform/255)

# Without complex numbers <br>

In [19]:
plt.figure('img_without_boundary')
plt.imshow(img_without_boundary/255)
y_img, x_img, z_img = img_without_boundary.shape
print(y_img, x_img, z_img)

31 28 3


In [16]:
# setting the x - coordinates 
x_plot = np.arange(-1*x_img/2, x_img/2 , 0.1) 
# print(x_plot)
# setting the corresponding y - coordinates 
# potting the points 
f_plot = x_plot
plt.figure('plot')
plt.plot(x_plot, f_plot)
plt.grid(True)
# function to show the plot 
plt.show() 

In [17]:
import scipy.integrate as integrate
x = sp.Symbol('x')  
f = x**2
# shift half pixel to the right 
f_derviate = sp.diff(f) 
print(type(f_derviate)) 
# func = sp.Symbol(str(f_derviate))
func = f_derviate
print(func)
print(type(func))

u = np.zeros((10, 3), dtype=sp.Symbol)

def get_function(x):
    return x

tmp = integrate.quad(lambda x: sp.Symbol('f_derviate'),0,1)
print(type(tmp))

<class 'sympy.core.mul.Mul'>
2*x
<class 'sympy.core.mul.Mul'>


TypeError: can't convert expression to float

In [ ]:
# https://www.youtube.com/watch?v=QAqadzwo19Q
# a = quad(lambda x: sqrt(x**2+1),0,5)
# print(type(x))
# print(a)
# print(type(float(a[0])))

In [ ]:
index = np.array(np.zeros((1,40)))
for i in range(0,math.ceil(x_img/2)):
    function = lambda x: d
    tmp = quad(get_function(x),0,2)
    
    
#     index[0,i] = round(float(tmp[0]))
#     print(index[0,i])
#     if index[0,i] >= math.ceil(x_img/2):
#         break

#     print(index[0,i])
#     print(type(index[0,0]))

print(tmp[0])

In [ ]:
max_value = i-1 
print(max_value)
print(x_img/2)

In [ ]:
empty_field = np.array(np.zeros((y_img, x_img, z_img)))
print(empty_field.shape)
for i in range(0,y_img):
    for j in range(0,2*max_value):
        for z in range(0,z_img):
            l = math.floor(j-max_value)
#             print(l)
            tmp = quad(lambda x: sqrt(x**2+1),0,l)
            tmp = round(tmp[0])
            print(tmp)
#             print(j)
            if img_without_boundary[i,j,z] > 0:
                    empty_field[i,tmp+max_value,z] = img_without_boundary[i,j,z]
plt.figure('after_transforamtion')
plt.imshow(empty_field/255)

In [ ]:
b = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
b[1:7:2]
np.array([1, 3, 5])
b[-2:10]
np.array([8, 9])
b[-3:3:-1]
np.array([7, 6, 5, 4])

In [ ]:
print(img_without_boundary[:,i,:])

In [ ]:
# https://www.math.ubc.ca/~pwalls/math-python/differentiation/#arc-length
def arc_length(f,a,b,h=0.001,N=1000):
    '''Approximate the ac length of y=f(x) from x=a to x=b.

    Parameters
    ----------
    f : (vectorized) function of one variable
    a,b : numbers defining the interval [a,b]
    h : step size to use in difference formulas
    N : number of subintervals in trapezoid method

    Returns
    -------
    Approximation of the integral \int_a^b \sqrt{1 + (f'(x))^2} dx
    representing the arc length of y=f(x) from x=a to x=b.
    '''
    x = np.linspace(a,b,N+1)
    y = f(x)

    # Compute central difference formula for x_k for 1 <= k <= N-1
    h = np.min([h,(b-a)/N]) # Make sure that h is smaller than the size of the subintervals
    x_interior = x[1:-1]
    df_interior = (f(x_interior + h) - f(x_interior - h))/(2*h)

    # Use forward/backward difference formula at the endpoints
    df_a = (f(a + h) - f(a))/h
    df_b = (f(b) - f(b - h))/h
    df = np.hstack([[df_a],df_interior,[df_b]])

    # Compute values of the integrand in arc length formula
    y = np.sqrt(1 + df**2)

    # Compute the integral
    L = spi.trapz(y,x)

    return L

In [ ]:
a = 1
result = arc_length(lambda x: a*x**2,0,1,h=10**(-10),N=10**5)
print(result)

In [ ]:
def function(x)
    return x**2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Fixing random state for reproducibility
np.random.seed(123456789)
y_max = 50

fig, ax = plt.subplots()
line, = ax.plot(np.random.rand(y_max))
ax.set_ylim(0, 1)


def update(data):
    line.set_ydata(data)
    return line,


def data_gen():
    while True:
        yield np.random.rand(y_max)

ani = animation.FuncAnimation(fig, update, data_gen, interval=100)
plt.show()

# Transformation from internet modified <br>

In [47]:
# https://github.com/mdbloice/Augmentor
import Augmentor as Aug
from PIL import Image, ImageFont, ImageDraw


In [29]:
path_folder = 'C:/Users/User/Desktop/test/generateImages' # Bilder Pfad
path_folder, dirs, files = next(os.walk(path_folder))
number_images = len(files)
# save_folder = 'C:/Users/User/Desktop/test/Augmentor/GenerateImages/'
print(path_folder)
print(number_images)
 

C:/Users/User/Desktop/test/generateImages
1


In [181]:
for i in range(1000):    
    skew_magnitude = np.random.uniform(0.1, 1.0) # 0.1-1.0
    new_size_ratio = np.random.uniform(0.7, 7.0) # depends on initanal size (20-250)
    rd_grid_with = np.random.randint(2,10) # 2-10
    rd_grid_hight = np.random.randint(2,10) # 2-10
    rd_magnitude = np.random.randint(1,10) # 1-10

    p = Aug.Pipeline(path_folder, output_directory='output', save_format="BMP")
    p.random_distortion(probability=1, grid_width=rd_grid_with, grid_height=rd_grid_hight, magnitude=rd_magnitude)
    p.rotate(probability=1, max_left_rotation=25, max_right_rotation=25) # >= 25° for botch
    p.skew(probability=0.3, magnitude=skew_magnitude)
    p.shear(probability=0.3, max_shear_left=25, max_shear_right=25) # >= 25° for botch
    p.resize(probability=1, width=int(size_x*new_size_ratio) , height=int(size_y*new_size_ratio), resample_filter='BICUBIC')
    p.sample(1)
    
print(type(p))
# print(b.shape)
# img_size_uint8 = Image.fromarray(np.uint8(img_mini))
# img_size_uint8.save(save_folder +

Initialised with 1 image(s) found.
Output directory set to C:/Users/User/Desktop/test/generateImages\output.

Processing <PIL.Image.Image image mode=RGBA size=169x169 at 0x15720588>: 100%|█| 1/1 [00:00<00:00,  5.88 Samples/s]


Initialised with 1 image(s) found.
Output directory set to C:/Users/User/Desktop/test/generateImages\output.

Processing <PIL.Image.Image image mode=RGBA size=1228x1228 at 0x15720588>: 100%|█| 1/1 [00:00<00:00,  5.43 Samples/s]


Initialised with 1 image(s) found.
Output directory set to C:/Users/User/Desktop/test/generateImages\output.

...

Initialised with 1 image(s) found.
Output directory set to C:/Users/User/Desktop/test/generateImages\output.

Processing <PIL.Image.Image image mode=RGBA size=324x324 at 0x177078D0>: 100%|█| 1/1 [00:00<00:00,  4.97 Samples/s]


Initialised with 1 image(s) found.
Output directory set to C:/Users/User/Desktop/test/generateImages\output.

Processing <PIL.Image.Image image mode=RGBA size=988x988 at 0x17707C18>: 100%|█| 1/1 [00:00<00:00,  7.75 Samples/s]


<class 'Augmentor.Pipeline.Pipeline'>


In [26]:
# https://www.kaggle.com/bguberfain/elastic-transform-for-data-augmentation
# https://www.ritchieng.com/machine-learning-photo-ocr/

In [130]:
# def iterate():
#     img_output = np.zeros((rows+512, cols+512))
#     for i in range(rows):
#         for j in range(cols):
#             offset_x = int(15.0 * np.sin(6*np.pi*i/180))
#             offset_y = 0 #int(d * np.sin(e * 3.14 * 1*(i+f)))
# #             print(offset_x,offset_y)
# #             print(i+offset_y+h,j+offset_x+d)

#             img_output[i+offset_y+200,j+offset_x+200] = img[i,j]
            
# #             plt.figure('Input')
# #             plt.imshow(img, cmap='gray')
# #             plt.figure('Vertical wavecols')
# #             plt.imshow(img_output, cmap='gray')

#     plt.figure('bla')
#     plt.imshow(img_output/255)

#     img_rect = deepcopy(img_output)
#     x, y, w, h = cv2.boundingRect(img_rect)
#     # print(x, y, w, h)
#     x_min = x-1
#     y_min = y-1
#     x_max = x+w
#     y_max = y+h
#     cv2.rectangle(img_rect, (x_min, y_min), (x_max, y_max), (0, 255, 0), 0)   # img wird überschrieben
#     plt.figure('img_rect')
#     plt.imshow(img_rect/255)


    
#     img_save = image_rect(img_output)
#     img_size_uint8 = Image.fromarray(np.uint8(img_output))
#     img_size_uint8.save(path + str(a)+'_'+str(b)+'_'+str(c)+'_'+str(d)+'.bmp')
                    
#         img_size_uint8 = Image.fromarray(np.uint8(img_size))
#         img_size_uint8.save(path_save_image + files[i])

In [59]:
# https://subscription.packtpub.com/book/application_development/9781785283932/1/ch01lvl1sec16/image-warping
# img_2D = img[:,:,0]
a = 15
b = 3
c = 1
d = 1

rows, cols = img_2D.shape 
img_output = np.zeros((200, 200))
img_output_3D = np.zeros((200, 200,3))
print(img_output.shape)
print(img.shape)
for i in range(rows):
    for j in range(cols):
        offset_x = int(a * np.sin(2*np.pi*i*b/180))
        offset_y = 0 # int(c * np.sin(2*np.pi*j*d/180))
#       print(offset_x,offset_y)
#       print(i+offset_y+h,j+offset_x+d)
        img_output[i+offset_y+50, j+offset_x+50] = img_2D[i,j]

# plt.figure('img_output')
# plt.imshow(img_output)
# print(type(img_output))

img_output_3D[:,:,0] = img_output
img_output_3D[:,:,1] = img_output
img_output_3D[:,:,2] = img_output

img_rect = deepcopy(img_output_3D)
img_rect = img_rect.astype(np.uint8)
x, y, w, h = cv2.boundingRect(img_rect[:,:,0])
# print(x, y, w, h)
x_min = x-1
y_min = y-1
x_max = x+w
y_max = y+h
cv2.rectangle(img_rect, (x_min, y_min), (x_max, y_max), (0, 255, 0), 0)   # img wird überschrieben
# plt.figure('img_rect')
# plt.imshow(img_rect/255)

img_without_boundary = img_rect[y:y_max, x:x_max,:]
plt.figure('img_without_boundary'+str(a)+'_'+str(b)+'_'+str(c)+'_'+str(d))
plt.imshow(img_without_boundary/255)


(200, 200)
(50, 50, 3)


In [65]:
# https://subscription.packtpub.com/book/application_development/9781785283932/1/ch01lvl1sec15/projective-transformations

import cv2
import numpy as np

rows, cols = img.shape[:2]

# src_points = np.float32([[0,0], [cols-1,0], [0,rows-1], [cols-1,rows-1]])
# dst_points = np.float32([[0,0], [cols-1,0], [int(0.1*cols),rows-1], [int(0.5*cols),rows-1]]) 
src_points = np.float32([[0,0], [0,rows-1], [cols/2,0], [cols/2,rows-1]])
dst_points = np.float32([[0,100], [0,rows-101], [cols/2,0], [cols/2,rows-1]])
projective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)
img_output = cv2.warpPerspective(img, projective_matrix, (cols,rows))

plt.figure('Input')
plt.imshow(img)
plt.figure('Output')
plt.imshow(img_output)